In [1]:
import os
import sys
import numpy as np

from mc_manager import *

%load_ext autoreload
%autoreload 2

import ptvsd
ptvsd.enable_attach()

('0.0.0.0', 5678)

In [27]:
ss = SeedSequence(123456)

In [28]:
ss

SeedSequence(
    entropy=123456,
)

In [2]:
scen_block_size = 100000
num_rns = 10000
num_scens = 100000
num_scens_for = 100

def get_res():

    base_bit_gen = np.random.PCG64(1234)

    rn_locs = {str(i):i for i in range(num_rns)}
    mcm = MonteCarloManager(base_bit_gen, num_scens, rn_locs)
    kw_res = {}
    n_s_b = [num_scens_for] * int(num_scens / num_scens_for)
    for b in range(len(n_s_b)):
        n_scen_b = n_s_b[b]

        rn_shuffled = list(rn_locs.keys())
        np.random.shuffle(rn_shuffled)
        #print(rn_shuffled[:10])

        for rn in rn_shuffled:
            if b == 0:
                kw_res[rn] = []
            rng_rn = mcm.get_rng(rn)
            rx = rng_rn.standard_normal(n_scen_b)
            #kw_res[rn].append(rx)
    
        if b % 100 == 0:
            print(b)
    #for k in kw_res:
    #    kw_res[k] = np.hstack(kw_res[k])

    return kw_res

def get_res_simple():
    gen = Generator(np.random.PCG64(1234))
    rn_locs = {str(i):i for i in range(num_rns)}
    kw_res = {}
    n_s_b = [num_scens_for] * int(num_scens / num_scens_for)
    for b in range(len(n_s_b)):
        n_scen_b = n_s_b[b]

        rn_shuffled = list(rn_locs.keys())
        np.random.shuffle(rn_shuffled)
        #print(rn_shuffled[:10])

        for rn in rn_shuffled:
            if b == 0:
                kw_res[rn] = []
            #rng_rn = mcm.get_rng(rn)
            rx = gen.standard_normal(n_scen_b)
    
        if b % 100 == 0:
            print(b)
    #for k in kw_res:
    #    kw_res[k] = np.hstack(kw_res[k])

    return kw_res    


In [4]:
base_bit_gen = np.random.PCG64(1234)

rn_locs = {str(i):i for i in range(num_rns)}
mcm = MonteCarloManager(base_bit_gen, num_scens, rn_locs)

In [14]:
%%timeit
#g = mcm.get_rng('1234')
for i in range(5):
    x = mcm.get_rng('1234').random(1000)
#    print(x)

33.9 µs ± 479 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [15]:
%%timeit
for i in range(5):
    x = gen0.random(1000)

28.7 µs ± 1.51 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [16]:
np.random.PCG64(1234).state

{'bit_generator': 'PCG64',
 'state': {'state': 29299324949094424543410418505067287561,
  'inc': 107381791681050441119675421997145146149},
 'has_uint32': 0,
 'uinteger': 0}

In [18]:
gen = np.random.PCG64(23434)

In [19]:
gen.state

{'bit_generator': 'PCG64',
 'state': {'state': 90384696249803151054767011599127933927,
  'inc': 43937483307350155021151273082486432497},
 'has_uint32': 0,
 'uinteger': 0}

In [25]:
for j in range(1000):
    gen.random_raw(1000)
gen.state

{'bit_generator': 'PCG64',
 'state': {'state': 166922040997625702861024576476815485991,
  'inc': 43937483307350155021151273082486432497},
 'has_uint32': 0,
 'uinteger': 0}

In [26]:
for j in range(1000):
    gen.jumped()
gen.state

{'bit_generator': 'PCG64',
 'state': {'state': 166922040997625702861024576476815485991,
  'inc': 43937483307350155021151273082486432497},
 'has_uint32': 0,
 'uinteger': 0}

In [22]:
mt = np.random.MT19937(1234)

In [23]:
gen

{'bit_generator': 'MT19937',
 'state': {'key': array([2147483648, 1602421836,  216676975, 2698621808,  507409703,
         2825158027,    3613074, 1752862323, 3963110497,   14931180,
         1016650013, 2259422597, 1544137552, 2160664294, 3920520203,
         3420798572,  743355607, 3968426182,  795979252, 3603776228,
         3964897866,  653764760, 3440141737, 3345345004,  761650748,
         2058157603,  137812659,  938474566, 3240371069, 2953889471,
         1406973561, 2779671463, 4099737540, 4134408353, 1182769146,
         3680250613, 4147700769, 3916251010, 2939039627, 1121760981,
         2595144951,  496853650,   28449591, 1689075076, 1243809658,
         4288078009, 3679049708,  465295410,  976188598, 1851078938,
         2467623569,  204271109, 3205476630, 2092708819, 1964216502,
          567631196, 3074911262,  978032068, 2915351617, 3828891025,
         1075112525, 3742003877, 3752032006, 3376792908, 3917398247,
         3717937748,  131346769, 4276879685, 2026582482, 4

In [153]:
%%time
x = get_res()

0
100
200
300
400
500
600
700
800
900
CPU times: user 49.5 s, sys: 239 ms, total: 49.7 s
Wall time: 49.8 s


In [150]:
%%time
x = get_res_simple()

0
100
200
300
400
500
600
700
800
900
CPU times: user 29.2 s, sys: 30.2 ms, total: 29.2 s
Wall time: 29.3 s


In [3]:
gen0 = Generator(np.random.PCG64(1234))
gen1 = Generator(np.random.PCG64(1234))

x0 = np.array([gen0.standard_normal() for i in range(num_scens)])
x1 = gen1.standard_normal(num_scens)

In [125]:
np.max(np.abs(x0 - x1))

0.0

In [127]:
sq1 = np.random.SeedSequence()

In [133]:
x = sq1.spawn(100000)

In [151]:
%%time
for i in range(100000):
    #gen0.bit_generator.advance(i)
    bit_get = PCG64()
    #gen0.bit_generator.state = gen1.bit_generator.state
    #Generator(gen0.bit_generator)

CPU times: user 4.91 s, sys: 1.55 s, total: 6.46 s
Wall time: 6.47 s


In [60]:
rn_shuffled = list(rn_locs.keys())
np.random.shuffle(rn_shuffled)

In [58]:
id = '8924'
np.max(np.abs(res1[id] - res2[id]))

0.0

In [5]:
for rn in rn_locs:
    mcm.get_rng(rn)

In [31]:
%%time
for rn in rn_locs:
    mcm.get_rng(rn).standard_normal(10000)

CPU times: user 1.19 s, sys: 0 ns, total: 1.19 s
Wall time: 1.2 s


In [32]:
%%time
rng.standard_normal((10000 * len(rn_locs)))

CPU times: user 1.26 s, sys: 393 ms, total: 1.65 s
Wall time: 1.69 s


array([-1.23906745,  1.50086196,  0.64057142, ...,  0.26374408,
        2.31379874, -1.30655306])

In [ ]:
%time


In [7]:
rng = Generator(base_bit_gen)

In [8]:
rng.standard_normal(100)

array([-1.60383681,  0.06409991,  0.7408913 ,  0.15261919,  0.86374389,
        2.91309922, -1.47882336,  0.94547297, -1.66613546,  0.34374458,
       -0.51244371,  1.32375896, -0.86028019,  0.5194932 , -1.26514372,
       -2.15913901,  0.43473395,  1.73328932,  0.52013416, -1.00216579,
        0.26834554,  0.7671747 ,  1.19127203, -1.15741081,  0.6962794 ,
        0.35138369, -0.03241508,  0.01318158, -0.67924997, -0.62053203,
        1.33121422,  0.25883851, -0.48148392, -2.49178962, -0.87656377,
       -0.50550913, -1.28312917, -1.33032842,  0.82599258, -0.24721501,
       -1.69970612, -1.33515287, -0.29963889,  1.1148069 , -1.50640885,
        1.59011208, -0.48732518, -1.71110219,  0.51309021,  1.43709192,
       -0.22180411,  0.6488165 , -0.31789119, -0.01097826,  1.6654165 ,
        0.89578643, -1.20260119,  2.79962711, -1.02119625,  0.848107  ,
        0.49808261, -0.08444164,  0.20249333, -0.16380578,  0.83705971,
       -0.71243966, -1.17414987,  0.47526795,  1.73739481, -0.13

In [32]:
base_bit_gen.state = {'bit_generator':'PCG64', 'state': {'state': 285989491001305928703026650751981614233,
  'inc': 107381791681050441119675421997145144149},
 'has_uint32': 0,
 'uinteger': 0}

In [33]:
base_bit_gen.state

{'bit_generator': 'PCG64',
 'state': {'state': 285989491001305928703026650751981614233,
  'inc': 107381791681050441119675421997145144149},
 'has_uint32': 0,
 'uinteger': 0}